In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import math

In [2]:
def sep_by_class(X_train, y_train):
    labels = []
    mean = []
    std = []
    X_train = pd.DataFrame(X_train)
    y_train = pd.Series(y_train)
    arr_of_labels = (y_train).unique()
    num_of_class = len(pd.Series(y_train).unique())
    #print("There are:", num_of_class, "classes of the y label")
    #print("The labels are:", arr_of_labels)
    for i in arr_of_labels:
        #print("Row seperated by label of:", i)
        labels.append(i)
        #print(X_train[(y_train == i)])
        #print("Mean of the value:")
        mean.append((X_train[(y_train == i)]).mean())
        #print("Standard deviation: ")
        std.append((X_train[(y_train == i)]).std())
    return labels, mean, std

In [3]:
def calculateProbability(x, mean, stdev):
    stdev = np.array(stdev)
    exponent = (np.exp((np.array(-(x - mean)**2))/np.array((2*(stdev**2)))))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

In [4]:
def calculate_class_probability(labels, mean, std, x):
    #print("There are:", len(x), "features in X_test")
    class_probs = []
    for i in range(len(labels)):
        #print(labels[i])
        #print(x)
        #print(mean[i])
        #print(std[i])
        probs = np.prod(calculateProbability(x, (mean[i]), (std[i])))
        #print(probs)
        #print(probs)
        class_probs.append(probs)
    return class_probs

In [5]:
def cond_prob(feat_1, feat_2):
    feat_1 = pd.Series(feat_1)
    feat_2 = pd.Series(feat_2)
    summation = []
    #print(feat_1)
    #print(feat_2)
    #print(feat_1.value_counts().divide(len(feat_1)))
    #print(feat_1)
    for i in (feat_1.unique()):
        #print("Given the value of:", i, "from feature 1")
        #print((feat_1[feat_1 == i]))
        #print((feat_1[feat_1 == i]).index)
        #print(feat_2[(feat_1[feat_1 == i]).index])
        #print(feat_1 == i)
        #print(i)
        cond_proba = (feat_2[(feat_1[feat_1 == i]).index]).value_counts().divide(len(feat_1))
        #print(cond_proba)
        proba = (feat_1.value_counts().divide(len(feat_1)))
        #print("These are the values for the probability of feature 1:")
        #print(proba)
        summation.append(proba * cond_proba)
        #print(summation)
    #print(pd.DataFrame(summation))
    return ((pd.DataFrame(summation)).sum(axis = 0))

In [6]:
def NB_predict(X_train, y_train, X_test, y_lag= [], weight = 'yes'):
    labels, mean, std = sep_by_class(X_train, y_train)
    y_train = pd.Series(y_train)
    class_probs = []
    if weight == 'yes':
        prob_of = ((y_train.value_counts()).divide(len(y_train)))
    elif weight == 'markov':
        prob_of = cond_prob(y_lag, y_train)
    elif weight == 'no':
        prob_of = 1
    #print(prob_of)
    """
    for i in (y_lag_test).values:
        prob_of = prob_of.loc[i]
    """
    X_test = pd.DataFrame(X_test)
    for i in range(len(X_test)):
            #print(i)
            #print(X_test.iloc[i,:])
        class_probs.append(prob_of*(pd.Series(calculate_class_probability(labels,mean, std, X_test.iloc[i,:]), labels)))
    predicted_labels = []
    #print(class_probs)
    for i in range(len(class_probs)):
        predicted_labels.append((class_probs[i]).idxmax())
    return predicted_labels

In [7]:
#Train up up to 168 then start forcasting in a window after 168
def window_tester(X, y, init_train_indx):
    lst_of_predicted = []
    lst_of_actual = []
    X = np.array(X)
    y = np.array(y)
    #print(X[:,:])
    for i in range(init_train_indx + 1,len(y)):
        X_train = X[:i-1,:]
        y_train = y[:i-1]
        X_test = X[[i][:]]
        y_test = y[i]
        #clf.fit(X = X_train, y = y_train)
        y_pred = NB_predict(X_train, y_train, X_test)
        #print(y_pred)
        lst_of_predicted.append(y_pred)
        lst_of_actual.append(y_test)
        #print("This is y_pred:   ", y_pred)
        #print("This is y_actual: ", y_test)
    return lst_of_predicted, lst_of_actual
    

In [34]:
#path = "C://Users//James//Downloads//macro_data.csv"
path = "C://Users//James//Downloads//macro_data_transformed.csv"

In [38]:
df = pd.read_csv(path, index_col='Unnamed: 0')
df = df.drop(1)

In [40]:
print(df.head())

    sasdate  NAPMPI  PAYEMS  T10YFFM   S.P.500  rec_dates
2  1/1/1959    70.7   207.0     1.54 -0.015400        0.0
3  2/1/1959    70.3   329.0     1.53  0.025062        0.0
4  3/1/1959    71.2   304.0     1.19  0.016599        0.0
5  4/1/1959    72.1   229.0     1.16  0.014949        0.0
6  5/1/1959    73.9   130.0     1.41 -0.008664        0.0


In [41]:
features = ['NAPMPI', 'PAYEMS', 'T10YFFM','S.P.500']
target = 'rec_dates'
#month of is it a recession
#y = df[target]

#next month is it a recession
horizon = [-1,-3,-6,-9,-12]
from sklearn.metrics import confusion_matrix
for val in horizon:
    X = df[features]
    print(val)
    y = df['rec_dates'].shift(val)
    X = X.iloc[:val,:]
    y = y.iloc[:val]
    y_pred, y_true = window_tester(X,y,168)
    TP = confusion_matrix(pd.DataFrame(y_true),pd.DataFrame(y_pred))[1][0]
    FP = confusion_matrix(pd.DataFrame(y_true),pd.DataFrame(y_pred))[1][1]
    TN = confusion_matrix(pd.DataFrame(y_true),pd.DataFrame(y_pred))[0][0]
    FN = confusion_matrix(pd.DataFrame(y_true),pd.DataFrame(y_pred))[0][1]
    print("Overall accuracy:")
    print((TP +TN)/(TP + TN + FP + FN))
    print("Precision Accuracy")
    print(TP/(TP + FP))
    print("Recall Accuracy")
    print(TP/(TP + FN))
    print("True Negative Rate")
    print(TN/(TN + FP))
    print("Negative Predictive Value")
    print(TN/(TN + FN))

-1
Overall accuracy:
0.8185328185328186
Precision Accuracy
0.1111111111111111
Recall Accuracy
0.21052631578947367
True Negative Rate
0.8666666666666667
Negative Predictive Value
0.9327354260089686
-3
Overall accuracy:
0.8410852713178295
Precision Accuracy
0.2777777777777778
Recall Accuracy
0.4
True Negative Rate
0.8884120171673819
Negative Predictive Value
0.9324324324324325
-6
Overall accuracy:
0.884990253411306
Precision Accuracy
0.4444444444444444
Recall Accuracy
0.6274509803921569
True Negative Rate
0.9134199134199135
Negative Predictive Value
0.9569160997732427
-9
Overall accuracy:
0.9235294117647059
Precision Accuracy
0.6527777777777778
Recall Accuracy
0.7704918032786885
True Negative Rate
0.9443207126948775
Negative Predictive Value
0.9680365296803652
-12
Overall accuracy:
0.9467455621301775
Precision Accuracy
0.782608695652174
Recall Accuracy
0.8181818181818182
True Negative Rate
0.9659863945578231
Negative Predictive Value
0.9726027397260274
